In [32]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
import imageio
from skimage.transform import resize

In [95]:
import warnings
warnings.filterwarnings('ignore', message="The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.")

In [114]:
from PIL import Image
from ipywidgets import interact

def display_field(field):
    return Image.fromarray(field)

def display_sequence(images):
    images_zerod = images-images.min()
    images_normd = images_zerod / images_zerod.max()
    images_255 = np.uint8(images_normd * 255)
    
    def _show(frame=(0, len(images_255) - 1)):
        return display_field(images_255[frame])
    
    return interact(_show)

In [2]:
h5f = h5py.File('sim_result.h5', 'r')

In [36]:
# from http://flatplanet.sourceforge.net/maps/natural.html
earth_img = imageio.imread('earthicefree.jpg').astype(np.float32) / 255

In [16]:
def get_data(angle, file):
    data = file[angle]
    arr = np.empty(data.shape)
    data.read_direct(arr)
    return arr

In [108]:
def gray_to_heat(grayscale_image):
    cmap = plt.get_cmap('plasma')
    heat = cmap(grayscale_image)
    return heat[...,:3]
    
def compose_images(earth, light):
    light_rgb = gray_to_heat(light)
    light_rgb = resize(light_rgb, earth.shape)
    combined = earth*0.25+light_rgb*0.75
    return np.clip(combined, 0, 1)

In [90]:
light_samples = []

for i in range(91):
    light_samples.append(get_data(str(i), h5f))
    print('loaded {}/90'.format(i), end='\r')

## Mean Light per Rotational Period

In [106]:
mean_light = np.empty((len(light_samples), *light_samples[0].shape[1:]))

for i,s in enumerate(light_samples):
    mean_light[i] = s.mean(axis=0)
    print('processed {}/90'.format(i), end='\r')
    
mean_light *= 1/mean_light.max()

In [118]:
mean_images = np.empty((len(light_samples), *earth_img.shape))

for i in range(len(mean_light)):
    mean_images[i] = compose_images(earth_img, mean_light[i])
    print('processed {}/90'.format(i), end='\r')

In [119]:
display_sequence(mean_images);

interactive(children=(IntSlider(value=45, description='frame', max=90), Output()), _dom_classes=('widget-inter…

In [133]:
for i in range(len(mean_images)):
    imageio.imwrite('./images_mean/{}.jpg'.format(i), (mean_images[i]*255).astype(np.uint8))

## Variance across Rotational Periods

In [122]:
var_light = np.empty((len(light_samples), *light_samples[0].shape[1:]))

for i,s in enumerate(light_samples):
    var_light[i] = s.std(axis=0)
    print('processed {}/90'.format(i), end='\r')
    
var_light *= 1/var_light.max()

var_images = np.empty((len(light_samples), *earth_img.shape))

for i in range(len(mean_light)):
    var_images[i] = compose_images(earth_img, var_light[i])
    print('processed {}/90'.format(i), end='\r')

In [123]:
display_sequence(var_images);

interactive(children=(IntSlider(value=45, description='frame', max=90), Output()), _dom_classes=('widget-inter…

In [134]:
for i in range(len(var_images)):
    imageio.imwrite('./images_var/{}.jpg'.format(i), (var_images[i]*255).astype(np.uint8))

## Max and Min Light per Rotational Period

In [128]:
max_light = np.empty((len(light_samples), *light_samples[0].shape[1:]))
min_light = np.empty((len(light_samples), *light_samples[0].shape[1:]))

for i,s in enumerate(light_samples):
    max_light[i] = s.max(axis=0)
    min_light[i] = s.min(axis=0)
    print('processed {}/90'.format(i), end='\r')
    
max_light *= 1/max_light.max()
min_light *= 1/min_light.max()

max_images = np.empty((len(light_samples), *earth_img.shape))
min_images = np.empty((len(light_samples), *earth_img.shape))

for i in range(len(mean_light)):
    max_images[i] = compose_images(earth_img, max_light[i])
    min_images[i] = compose_images(earth_img, min_light[i])
    print('processed {}/90'.format(i), end='\r')

In [121]:
display_sequence(max_images);

interactive(children=(IntSlider(value=45, description='frame', max=90), Output()), _dom_classes=('widget-inter…

In [129]:
display_sequence(min_images);

interactive(children=(IntSlider(value=45, description='frame', max=90), Output()), _dom_classes=('widget-inter…

In [135]:
for i in range(len(max_images)):
    imageio.imwrite('./images_max/{}.jpg'.format(i), (max_images[i]*255).astype(np.uint8))

In [136]:
for i in range(len(min_images)):
    imageio.imwrite('./images_min/{}.jpg'.format(i), (min_images[i]*255).astype(np.uint8))